In [1]:
import numpy as np

In [2]:
def init_layer_weights(layerDefinition):
    input_size = layerDefinition[0];
    output_size = layerDefinition[1];
    weights = np.random.rand(input_size, output_size) - 0.5;
    bias = np.random.rand(1, output_size) - 0.5;
    return weights, bias

def mse(y_true, y_pred):
    return np.mean(np.power(y_true-y_pred, 2))

def mse_prime(y_true, y_pred):
    return 2*(y_pred-y_true)/y_true.size

In [3]:
def optimizer_SGD(weights, bias, input_error, output_error, weights_error, learning_rate):
    """ Stochastic Gradient Descent """
    weights_updated = weights -  (learning_rate * weights_error)
    bias_updated = bias - (learning_rate * output_error)
    updated_params = (weights_updated, bias_updated)
    return updated_params

In [4]:
def forward_propagation(input_data, layer):
    weights, bias = layer
    output = np.dot(input_data, weights) + bias
    return output

def backward_propagation(output_error, input_data, layer, learning_rate):
    weights, bias = layer
    input_error = np.dot(output_error, weights.T)
    weights_error = np.dot(input_data.T, output_error)
    #update
    updated_layer = optimizer_SGD(weights, bias, input_error, output_error, weights_error, learning_rate)
    return input_error, updated_layer

def forward_propagation_activation(input_data):
    output = np.tanh(input_data)
    return output

def backward_propagation_activation(output_error, input_data):
    activation_prime = 1 - np.tanh(input_data)**2
    final_output_error = activation_prime * output_error
    return final_output_error

In [5]:
def init_network(layerDefinitions):
    nlayers = len(layerDefinitions)
    network=[]
    for i in range(nlayers):
        if isinstance(layerDefinitions[i],str):
            network.append(layerDefinitions[i])
        else:
            network.append(init_layer_weights(layerDefinitions[i]))
    return network

def network_summary(network):
    """ print network summary """
    for i in range(len(network)):
        if isinstance(network[i],str):
            print("activation function:", network[i])
        else:
            for j in range(2):
                if j == 0:
                    print("layer", i, "weight shape",network[i][0].shape)
                elif j == 1:
                    print("layer", i, "bias shape",network[i][1].shape)
        

In [6]:
def forward_pass(X,network):
    output = X;
    nlayers = len(network)
    layer_outputs = []
    for i in range(nlayers):
        if isinstance(network[i],str):
            output = forward_propagation_activation(output)
            layer_outputs.append(output)
        else:
            output = forward_propagation(output,network[i])
            layer_outputs.append(output)
        
    return layer_outputs

In [7]:
def backward_pass(X,outputError,layer_outputs,network,learning_rate):
    back_output = outputError
    layer_inputs = []
    layer_inputs.append(X)
    for i in range(len(layer_outputs)-1):
        layer_inputs.append(layer_outputs[i])

    updated_layers=[]
    for i in reversed(range(len(network))):
        if isinstance(network[i],str):
            back_output = backward_propagation_activation(back_output,layer_inputs[i])
            updated_layers.append(network[i])
        else:
            back_output, updated_layer = backward_propagation(back_output,layer_inputs[i],network[i],learning_rate)
            updated_layers.append(updated_layer)
            
    return list(reversed(updated_layers))

In [8]:
def train(x_train, y_train, nsamples, network, epochs, learning_rate):
    updated_network = network
    for j in range(epochs):
        E = 0;
        for i in range(nsamples):
            x = x_train[i]
            y = y_train[i]
            #fwd pass
            layer_outputs = forward_pass(x,updated_network)
            #compute loss 
            E += mse(y,layer_outputs[-1])
            #compute loss derivative to initiate backward pass
            dE = mse_prime(y,layer_outputs[-1])
            #backward pass
            updated_network = backward_pass(x,dE,layer_outputs,updated_network,learning_rate)
            #avg error
            E /= nsamples
        print("Epoch number: ", j)
        print("Epoch error is: ", E)
        
    return updated_network

In [9]:
# load MNIST 
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder

data = np.load('mnist.npz')
x_train = data['x_train']
x_test = data['x_test']
y_train = data['y_train']
y_test = data['y_test']

# training data : 60000 samples
# reshape and normalize input data
x_train = x_train.reshape(x_train.shape[0], 1, 28*28)
x_train = x_train.astype('float32')
x_train /= 255

# same for test data : 10000 samples
x_test = x_test.reshape(x_test.shape[0], 1, 28*28)
x_test = x_test.astype('float32')
x_test /= 255

# encode output which is a number in range [0,9] into a vector of size 10
# e.g. number 3 will become [0, 0, 0, 1, 0, 0, 0, 0, 0, 0]
label_encoder = LabelEncoder()
onehot_encoder = OneHotEncoder(sparse=False)

int_encoded = label_encoder.fit_transform(y_train)
int_encoded = int_encoded.reshape(len(int_encoded),1)
y_train = onehot_encoder.fit_transform(int_encoded)

int_encoded = label_encoder.fit_transform(y_test)
int_encoded = int_encoded.reshape(len(int_encoded),1)
y_test = onehot_encoder.fit_transform(int_encoded)

/opt/anaconda3/lib/python3.8/site-packages/scipy/__init__.py:138: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.24.3)
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion} is required for this version of "


In [10]:
layerDefinitions = [(28*28,200),"tanh",(200,100),"tanh",(100,50),"tanh",(50,10),"tanh"]

In [11]:
network = init_network(layerDefinitions)

In [12]:
network_summary(network)

layer 0 weight shape (784, 200)
layer 0 bias shape (1, 200)
activation function: tanh
layer 2 weight shape (200, 100)
layer 2 bias shape (1, 100)
activation function: tanh
layer 4 weight shape (100, 50)
layer 4 bias shape (1, 50)
activation function: tanh
layer 6 weight shape (50, 10)
layer 6 bias shape (1, 10)
activation function: tanh


In [13]:
nsamples = 100
batch_size = 10
learning_rate = 0.1
epochs = 60
model = train(x_train, y_train, nsamples, network, epochs, learning_rate)

Epoch number:  0
Epoch error is:  0.004366828565349433
Epoch number:  1
Epoch error is:  0.001294824379965338
Epoch number:  2
Epoch error is:  0.0011610683799173319
Epoch number:  3
Epoch error is:  0.0006639491608130106
Epoch number:  4
Epoch error is:  0.0005278780653879004
Epoch number:  5
Epoch error is:  0.0006806960430999571
Epoch number:  6
Epoch error is:  0.00024412857840148307
Epoch number:  7
Epoch error is:  0.00024900871969231627
Epoch number:  8
Epoch error is:  0.00013069498032410251
Epoch number:  9
Epoch error is:  8.28368459748328e-05
Epoch number:  10
Epoch error is:  0.00012927796218030986
Epoch number:  11
Epoch error is:  0.00010228451711180768
Epoch number:  12
Epoch error is:  0.00012612723256731265
Epoch number:  13
Epoch error is:  6.65274832788406e-05
Epoch number:  14
Epoch error is:  0.00011078452531535253
Epoch number:  15
Epoch error is:  4.260316511373403e-05
Epoch number:  16
Epoch error is:  6.814707632893233e-05
Epoch number:  17
Epoch error is:  3.4

In [14]:
pred_results=[]
n_test_samples = 10000
for i in range(n_test_samples):
    pred = forward_pass(x_test[i], model)
    final_pred = np.argmax(pred[-1])
    truth_value = np.argmax(y_test[i])
    err = np.abs(final_pred - truth_value)
    pred_results.append(err)
    
percent_accuracy = (n_test_samples -  np.count_nonzero(np.asarray(pred_results)))/100
print("Percent Accuracy of NN: ", percent_accuracy)

Percent Accuracy of NN:  44.11


In [15]:
X = x_train[0]
Y = y_train[0]

In [16]:
allouts = forward_pass(X,network)

In [17]:
E = mse(Y,allouts[-1])
dE = mse_prime(Y,allouts[-1])

In [18]:
learning_rate = 0.1

In [19]:
dE.shape

(1, 10)

In [20]:
updated_network = backward_pass(X,dE,allouts,network,learning_rate)

In [21]:
network_summary(updated_network)

layer 0 weight shape (784, 200)
layer 0 bias shape (1, 200)
activation function: tanh
layer 2 weight shape (200, 100)
layer 2 bias shape (1, 100)
activation function: tanh
layer 4 weight shape (100, 50)
layer 4 bias shape (1, 50)
activation function: tanh
layer 6 weight shape (50, 10)
layer 6 bias shape (1, 10)
activation function: tanh


In [22]:
allouts2 = forward_pass(X,updated_network)

In [23]:
E = mse(Y,allouts2[-1])
dE = mse_prime(Y,allouts2[-1])
updated_network2 = backward_pass(X,dE,allouts2,updated_network,learning_rate)

In [24]:
network_summary(updated_network2)

layer 0 weight shape (784, 200)
layer 0 bias shape (1, 200)
activation function: tanh
layer 2 weight shape (200, 100)
layer 2 bias shape (1, 100)
activation function: tanh
layer 4 weight shape (100, 50)
layer 4 bias shape (1, 50)
activation function: tanh
layer 6 weight shape (50, 10)
layer 6 bias shape (1, 10)
activation function: tanh
